In [2]:
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations

Ising Hamiltonian:
$$
H = -J\sum_{j=0}^{d-1}\vec{S}_{j}\cdot\vec{S}_{j+1} = -J/4 \sum_{j=0}^{d-1}(\sigma_{x}^{j}\sigma_{x}^{j+1}+\sigma_{y}^{j}\sigma_{y}^{j+1}+\sigma_{z}^{j}\sigma_{z}^{j+1})
$$
Lets consider the computational basis:
$$
\ket{\{s_{i}\}}
$$
where $i = 0,\ldots,d-1$ and $s_{i} = \pm 1$.

We can distinguish ${n \choose k}$ states with $k$ spins directed upward. The same is for spins directed downward. How ca we find these states that are related by translation?  



In [38]:
class Ising_momentum:
    def __init__(self,dim):
        self.N = dim

    def group_strings(self,k):
        self.start_group = list(map(lambda el : list(el),combinations([i for i in range(self.N)],k)))
        group = []
        for i in range(len(self.start_group)):
            el = sorted(self.start_group[i])
            group.append([self.start_group[i]])
            el2 = el
            k = 0 
            while True:
                el2 = sorted([(i+1)%self.N for i in el2])
                if el2 == el:
                    k+=1
                    break
                else:
                    group[k].append(el2)
        return group



In [39]:
Ising = Ising_momentum(4)
Ising.group_strings(2)  

[[[0, 1],
  [1, 2],
  [2, 3],
  [0, 3],
  [1, 3],
  [0, 1],
  [1, 2],
  [2, 3],
  [2, 3],
  [0, 3],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 1],
  [1, 2]],
 [[0, 2]],
 [[0, 3]],
 [[1, 2]],
 [[1, 3]],
 [[2, 3]]]

In [31]:
tab = [3,4,2]


print(sorted(tab))

[2, 3, 4]
